In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import serial,time

size=4
model=tf.keras.models.load_model('own_yes_no.h5')

labels_dict = {1: 'with_mask', 0: 'without_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
webcam = cv2.VideoCapture(1)

ArduinoSerial=serial.Serial('com19',9600,timeout=0.1)
time.sleep(1)


while True:
    (rval, im) = webcam.read()
    im = cv2.flip(im, 1, 1)  
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))
    faces = classifier.detectMultiScale(mini)
    


    
    for f in faces:
        (x, y, w, h) = [v * size for v in f]  

        #sending coordinates to arduino
        string='X{0:d}Y{1:d}'.format((x+w//2),(y+h//2))
        print(string,' ',string.encode('utf-8'))
        ArduinoSerial.write(string.encode('utf-8'))

        face_img = im[y:y+h, x:x+w]
        resized = cv2.resize(face_img, (150, 150))
        normalized = resized/255.0
        reshaped = np.reshape(normalized, (1, 150, 150, 3))
        reshaped = np.vstack([reshaped])
        result = model.predict(reshaped)
        print(result[0][0]-0.45)

        label = int(np.round((result)[0][0]-0.40))
        # print(class_labels[label])
        
        res= 'P{0:d}'.format( label )
        ArduinoSerial.write(res.encode('utf-8'))
        
        print(result,"\n",string.encode('utf-8'),'\n',x, y, w, h,"\n",res,res.encode('utf-8'),'\n')
        
        
        #plot the center of the face
        cv2.circle(rval,(x+w//2,y+h//2),2,(0,255,0),2)

        cv2.rectangle(im, (x, y), (x+w, y+h), color_dict[label], 2)
        cv2.rectangle(im, (x, y-40), (x+w, y), color_dict[label], -1)
        cv2.putText(im, labels_dict[label], (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    if key == 27:  
        break
webcam.release()
cv2.destroyAllWindows()